<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/FaultRepair2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [5]:
# ─── SETUP FOR FAULT CONTROLLERS ───

import os, sys, subprocess

try:
    REPO_DIR = "/content/Cloud_Computing"
    LOGIC_DIR = os.path.join(REPO_DIR, "logic")

    # Step 1: Clone the repo if not present
    if not os.path.isdir(REPO_DIR):
        subprocess.run([
            "git", "clone", "-b", "develop",
            "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
            REPO_DIR
        ], check=True)

    # Step 2: Pull latest changes from develop
    subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

    # Step 3: Add logic folder to sys.path
    sys.path.append(LOGIC_DIR)

    # Step 4: Install dependencies
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio
    %pip install -q firebase
    %pip install requests beautifulsoup4
    %pip install -q matplotlib

    # Step 5: Import required notebooks
    from importnb import Notebook
    with Notebook():
        import Fault_controller as fc
        import repair_controller as rc
        import user_controller as uc

    print("✅ Fault setup completed successfully.")

except Exception as e:
    print("❌ Fault setup failed:", str(e))

✅ Fault setup completed successfully.


In [6]:
import gradio as gr
from datetime import datetime

# ---------- Sample Fault ----------
sample_fault = {
    "title": "Indoor Temperature above 35°C",
    "sensor": "Indoor Temperature Sensor A1",
    "severity": "High",
    "timestamp": "2025-05-19T14:07:00",
    "actions": [
        "Turn on the cooling system",
        "Inspect the indoor temperature sensor",
        "Check if the cooling system is functioning"
    ]
}

# ---------- Simulated Active Faults ----------
simulated_active_faults = [
    {
        "timestamp": "2025-05-19T23:00:00",
        "title": "Indoor Pressure dropped",
        "severity": "Low",
        "status": "Active"
    },
    {
        "timestamp": "2025-05-19T22:45:00",
        "title": "Outdoor Temperature spike",
        "severity": "High",
        "status": "Active"
    },
    {
        "timestamp": "2025-05-19T21:30:00",
        "title": "Humidity too high",
        "severity": "Medium",
        "status": "Active"
    }
]

# ---------- Helpers ----------
def get_severity_color(severity):
    return {
        "Low": "#fff176",
        "Medium": "#ffb74d",
        "High": "#ef5350"
    }.get(severity, "#fff176")

def format_datetime(iso_str):
    dt = datetime.fromisoformat(iso_str)
    return dt.strftime("%-d/%-m, %-I:%M %p")

def split_sensor(sensor):
    parts = sensor.split()
    return " ".join(parts[:2]), " ".join(parts[2:]) if len(parts) > 2 else ""

# ---------- Extracted ----------
severity_color = get_severity_color(sample_fault["severity"])
sensor1, sensor2 = split_sensor(sample_fault["sensor"])
time_str = format_datetime(sample_fault["timestamp"])
xp_value = {"Low": 50, "Medium": 100, "High": 200}.get(sample_fault["severity"], 50)

# ---------- Styles ----------
css = f"""
#main-fault {{
    background-color: {severity_color};
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#main-fault * {{
    color: white !important;
}}
#main-fault ul {{
  padding-left: 15px !important;
  margin-top: 4px;
  margin-bottom: 0;
}}

#repair-box {{
    background-color: white;
    color: black;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#repair-box * {{
    color: black !important;
}}
.repair-checkbox {{
  display: flex !important;
  flex-direction: row !important;
  gap: 20px;
  flex-wrap: wrap;
}}
.wrap default full svelte-ls20lj hide {{
  border: transperant !important;
}}
"""

# ---------- App ----------
with gr.Blocks(css=css) as demo:

    # -------- First Row ----------
    with gr.Row():
        with gr.Column(elem_id="main-fault", scale=1):
            gr.Markdown("### FAULT DETECTED")
            gr.Markdown(f"**{time_str}**")
            gr.Markdown(f"### {sensor1}")
            if sensor2:
                gr.Markdown(sensor2)
            gr.Markdown("#### Suggested Actions:")
            for action in sample_fault["actions"]:
                gr.Markdown(f"- {action}")

        with gr.Column(elem_id="repair-box", scale=1):
            gr.Markdown("## REPAIR CHALLENGE")
            gr.Markdown(f"Earn {xp_value} XP by fixing")
            gr.Markdown(f"{sensor2 or sample_fault['sensor']} within 10 min")
            gr.Markdown("#### Repair Steps")

            repair_checklist = gr.CheckboxGroup(
                choices=sample_fault["actions"],
                value=[],
                interactive=False,
                label="",
                elem_classes=["repair-checklist"]
            )

            start_btn = gr.Button("Start Repair", visible=True)
            finish_btn = gr.Button("Finish Repair", visible=False, interactive=False)
            status_output = gr.Textbox(label="System Message", interactive=False)

    # -------- Second Row ----------
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Active Faults")

            table_headers = gr.Dataframe(
                headers=["Time", "Sensor", "Severity", "Status"],
                value=[],
                row_count=0,
                col_count=(4, "fixed"),
                interactive=False,
                label=""
            )

    # ---------- Logic ----------
    def start_real_repair():
        sensor = sample_fault["sensor"]
        msg = rc.start_repair(sensor)
        return gr.update(interactive=True), gr.update(visible=False), gr.update(visible=True), msg

    def handle_checkbox_update(selected):
        all_checked = set(selected) == set(sample_fault["actions"])
        return gr.update(interactive=all_checked)

    def complete_real_repair():
        sensor = sample_fault["sensor"]
        msg = rc.complete_repair(sensor)
        return gr.update(visible=False), gr.update(value="Start Repair", visible=True, interactive=True), msg

    def load_active_faults():
        rows = [
            [
                format_datetime(f["timestamp"]),
                f["title"],
                f["severity"],
                f["status"]
            ] for f in simulated_active_faults
        ]
        return gr.update(value=rows)

    # ---------- Bindings ----------
    start_btn.click(
        fn=start_real_repair,
        outputs=[repair_checklist, start_btn, finish_btn, status_output]
    )

    repair_checklist.change(
        fn=handle_checkbox_update,
        inputs=repair_checklist,
        outputs=finish_btn
    )

    finish_btn.click(
        fn=complete_real_repair,
        outputs=[finish_btn, start_btn, status_output]
    )

    demo.load(fn=load_active_faults, outputs=table_headers)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c50519e0bac2503e4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
